<a href="https://colab.research.google.com/github/HariBejju/fine-tuning-basics/blob/main/finetuning_kanna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers


In [ ]:
pip install torch


In [ ]:
pip install --upgrade transformers

In [ ]:
from google.colab import userdata
hf_token=userdata.get('hf_token')

In [ ]:
# Install required libraries


# Import necessary libraries
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
from huggingface_hub import login
import torch

# Login to Hugging Face (replace with your token)
login(hf_token)

# Set the model id
model_id = "google/flan-t5-small"

# Check if CUDA is available and set the device accordingly
if torch.cuda.is_available():
    device = 'cuda'
    print("CUDA is available. Using GPU.")
    print(torch.cuda.get_device_name(0))
    model = T5ForConditionalGeneration.from_pretrained(model_id).to(device)  # Instantiate model to GPU
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")
    model = T5ForConditionalGeneration.from_pretrained(model_id)  # Instantiate model to CPU

#Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_id)

# Set pad token
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token to be the end of sequence token

# Create the pipeline
generating_pipeline = pipeline(task='text2text-generation', model=model, tokenizer=tokenizer, device=0 if device == 'cuda' else -1)

# Test the pipeline
print(generating_pipeline("vanakam bro", max_new_tokens=25))



CUDA is not available. Using CPU.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


[{'generated_text': '<pad> vanakam bro'}]


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
!pip install --upgrade transformers datasets


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving financial_dataset_100_companies.txt to financial_dataset_100_companies (1).txt
User uploaded file "financial_dataset_100_companies (1).txt" with length 22019 bytes


In [ ]:

data_path = '/content/financial_dataset_100_companies.txt'
with open(data_path, 'r') as file:
    lines = file.readlines()
print(lines[:5])

["Company 1 has an annual revenue of 5100 crores and a net profit of -505 crores. The company's debt-to-equity ratio is 0.6, with an EPS of -10.3. It has a market capitalization of 10200 crores and is considered Unstable.\n", "Company 2 has an annual revenue of 5200 crores and a net profit of 520 crores. The company's debt-to-equity ratio is 0.7, with an EPS of -10.6. It has a market capitalization of 10400 crores and is considered Unstable.\n", "Company 3 has an annual revenue of 5300 crores and a net profit of -515 crores. The company's debt-to-equity ratio is 0.8, with an EPS of 21.5. It has a market capitalization of 10600 crores and is considered Stable.\n", "Company 4 has an annual revenue of 5400 crores and a net profit of 540 crores. The company's debt-to-equity ratio is 0.9, with an EPS of -11.2. It has a market capitalization of 10800 crores and is considered Unstable.\n", "Company 5 has an annual revenue of 5500 crores and a net profit of -525 crores. The company's debt-to-e

In [ ]:
import re



# Join the list into a single string and strip leading/trailing whitespace
text_data = "".join(lines).strip()

# Define a regex pattern to extract data
pattern = re.compile(
    r"Company (?P<id>\d+) has an annual revenue of (?P<revenue>\d+) crores and a net profit of (?P<net_profit>-?\d+) crores\. "
    r"The company's debt-to-equity ratio is (?P<debt_to_equity>\d+\.\d+), with an EPS of (?P<eps>-?\d+\.\d+)\. "
    r"It has a market capitalization of (?P<market_cap>\d+) crores and is considered (?P<stability>\w+)\."
)

# Split the text into individual company entries
company_entries = text_data.split("\n")

# Extract data for each company
companies = []
for entry in company_entries:
    match = pattern.match(entry)
    if match:
        companies.append(match.groupdict())

# Generate question-answer pairs
qa_pairs = []
for company in companies:
    # Factual questions
    qa_pairs.append({
        "question": f"What is the annual revenue of Company {company['id']}?",
        "answer": f"The annual revenue of Company {company['id']} is {company['revenue']} crores."
    })
    qa_pairs.append({
        "question": f"What is the net profit of Company {company['id']}?",
        "answer": f"The net profit of Company {company['id']} is {company['net_profit']} crores."
    })
    qa_pairs.append({
        "question": f"What is the debt-to-equity ratio of Company {company['id']}?",
        "answer": f"The debt-to-equity ratio of Company {company['id']} is {company['debt_to_equity']}."
    })
    qa_pairs.append({
        "question": f"What is the EPS of Company {company['id']}?",
        "answer": f"The EPS of Company {company['id']} is {company['eps']}."
    })
    qa_pairs.append({
        "question": f"What is the market capitalization of Company {company['id']}?",
        "answer": f"The market capitalization of Company {company['id']} is {company['market_cap']} crores."
    })
    qa_pairs.append({
        "question": f"Is Company {company['id']} considered Stable or Unstable?",
        "answer": f"Company {company['id']} is considered {company['stability']}."
    })

# Comparative questions
qa_pairs.append({
    "question": "Which company has the highest annual revenue?",
    "answer": f"Company {max(companies, key=lambda x: int(x['revenue']))['id']} has the highest annual revenue."
})
qa_pairs.append({
    "question": "Which company has the lowest net profit?",
    "answer": f"Company {min(companies, key=lambda x: int(x['net_profit']))['id']} has the lowest net profit."
})

# Save the dataset as JSON
import json
with open("qa_dataset.json", "w") as f:
    json.dump(qa_pairs, f, indent=4)

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset, load_dataset
with open("qa_dataset.json", "r") as f:
    qa_pairs = json.load(f)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"question": [pair["question"] for pair in qa_pairs],
                             "answer": [pair["answer"] for pair in qa_pairs]})


In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["question"], max_length=512, padding="max_length", truncation=True)
    outputs = tokenizer(examples["answer"], max_length=512, padding="max_length", truncation=True)
    inputs["labels"] = outputs["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/602 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",  # Directory to save the model
    overwrite_output_dir=True,       # Overwrite the output directory
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=1,   # Batch size for training
    save_steps=500,                  # Save the model every 500 steps
    save_total_limit=2,              # Keep only the last 2 models
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    evaluation_strategy="no",        # Skip evaluation during training
    fp16=True,                       # Use mixed precision training
    learning_rate=5e-5,              # Learning rate
    report_to="none",                # Disable external reporting
    gradient_accumulation_steps=16   # Accumulate gradients
)

In [ ]:
trainer = Trainer(
    model=model,                     # The model to train
    args=training_args,              # The training arguments
    train_dataset=tokenized_dataset, # The training dataset
    tokenizer=tokenizer              # The tokenizer
)


<ipython-input-33-85a5d2ce1c37>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("Starting training...")
trainer.train()
print("Training complete!")

Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,28.145600


KeyboardInterrupt: 